This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [162]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [163]:
# First, import the relevant modules
import requests
from collections import OrderedDict

In [164]:
# Store the URL as a string. 
URL_AFX_X = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'

In [165]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
r = requests.get(URL_AFX_X+'?api_key='+API_KEY+'&limit=1')

In [166]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
r.json()

{'dataset_data': {'limit': 1,
  'transform': None,
  'column_index': None,
  'column_names': ['Date',
   'Open',
   'High',
   'Low',
   'Close',
   'Change',
   'Traded Volume',
   'Turnover',
   'Last Price of the Day',
   'Daily Traded Units',
   'Daily Turnover'],
  'start_date': '2000-06-07',
  'end_date': '2019-09-25',
  'frequency': 'daily',
  'data': [['2019-09-25',
    104.0,
    104.2,
    100.1,
    101.5,
    None,
    169893.0,
    17234866.0,
    None,
    None,
    None]],
  'collapse': None,
  'order': None}}

In [167]:
r.json()['dataset_data']['data']

[['2019-09-25',
  104.0,
  104.2,
  100.1,
  101.5,
  None,
  169893.0,
  17234866.0,
  None,
  None,
  None]]

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

# 1. Collect data from Frankfurt Stock Exchange for ticker AFX_X for 2017 .

In [168]:
r = requests.get(URL_AFX_X+'?api_key='+API_KEY+'&start_date=2017-01-01&end_date=2017-12-31')

# 2. Convert returned JSON object into Ordered Dictionary.

In [169]:
afx_od = OrderedDict()

# Pull headings/inner keys out of JSON.
afx_headings = r.json()['dataset_data']['column_names']

# Pull data values out of JSON.
afx_data = r.json()['dataset_data']['data']

# Build nested OrderedDict. Date is outer level key. Inner level keys are headings. Data is provided
# in reverse chronological order, which will be preserved by OrderedDict. 
for day in afx_data:
    afx_od[day[0]] = {afx_headings[x]: day[x] for x in range(1, len(afx_headings))}

# 3a. Calculate the Highest Opening Price during 2017.

In [170]:
# Eliminate None values from max input as they will throw an error. 
afx_open_max = max(a['Open'] for a in afx_od.values() if (a and a['Open'] is not None))
afx_open_max

53.11

# 3b. Calculate the Lowest Opening Price during 2017.

In [171]:
# Eliminate None values from min input as they will throw an error. 
afx_open_min = min(a['Open'] for a in afx_od.values() if (a and a['Open'] is not None))
afx_open_min

34.0

# 4. What was the Largest Daily Change during 2017 (based on High and Low price)?

In [172]:
# Eliminate observations with None values as they will throw an error in the calculation. 
afx_change_max = max((a['High'] - a['Low']) for a in afx_od.values() if (a and a['High'] is not None and a['Low'] is not None))
afx_change_max

2.8100000000000023

# 5. What was the Largest Closing Price Change Between Any Two Days in 2017?
#     Note: Assuming question means change between any two consecutive days market is open and prices are reported. 

In [173]:
last_close = None
max_change = 0
# Because data is in an OrderedDict and loaded in reverse chronological order, we just need to compare closing prices 
#    on adjacent keys. For days with None for Close and for first day, set daily change to zero. 
for vo in afx_od.values():
    if (last_close is None) or (vo['Close'] is None):
        change = 0
    else:
        change = abs(vo['Close'] - last_close)
    max_change = max(max_change, change)
    last_close = vo['Close']
    
max_change

2.559999999999995

# 6. Was was the Average Daily Trading Volume during 2017?

In [174]:
def avg(numbers, count):
    """Calculate average (mean) of a list of numbers, given the count. We need count in this case as the list
       is created by a generator, so has no len(). """
    if count == 0:
        return 0
    else:
        return float(sum(numbers)) / count

afx_volume_cnt = sum(1 for a in afx_od.values() if (a and a['Traded Volume'] is not None))
afx_volume_avg = avg((a['Traded Volume'] for a in afx_od.values() if (a and a['Traded Volume'] is not None)), afx_volume_cnt)
afx_volume_avg

89124.33725490196

# 7. What was the Median Trading Volume during 2017?

In [175]:
def median(sorted):
    """Calculate median of a sorted list of numbers. Return average of two middle numbers if even length, middle
       number if odd length."""
    if len(sorted) % 2 == 0:
        return ((sorted[(len(sorted) // 2) - 1] + sorted[len(sorted) // 2]) // 2)
    else:
        return sorted[len(sorted) // 2]

afx_sorted_volume = sorted(list(a['Traded Volume'] for a in afx_od.values() if (a and a['Traded Volume'] is not None)))
afx_median_volume = median(afx_sorted_volume)
afx_median_volume

76286.0